# Convert SAM 2D (x-y) output to DEPHY standards for SEA STARR

#### Code modified by Ehsan Erfani for UW-SAM, June 2022 @UW
#### Code created by Michael Diamond for NOAA-SAM, June 2022 @NOAA Boulder

In [1]:
#Import libraries
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import os
import datetime
#import nerodia as nero

### Constants

In [2]:
year  = str(datetime.datetime.now().year)
month = str(datetime.datetime.now().month)
day   = str(datetime.datetime.now().day)

### Converts SAM output to DEPHY standards

In [3]:
def create_DEPHY_2Doutput(input_path,output_path,caseid):
    """
    Convert SAM 2D (x-y) output to DEPHY/CF standards for SEA STARR.

    Parameters:
    -----------
    input_path : str
    Path to input data

    output_path : str
    Path in which to save output data

    caseid : str
    Case name
    """
    
    #Load data
    inp = xr.open_dataset(input_path)
    out = xr.Dataset()


    #=============
    ###Coordinates
    #=============

    #Time
    out['time'] = np.array((inp.time.values-226)*24*3600,dtype=int)
    out['time'].attrs = {'units' : 'seconds since 2017-08-14 00:00:00', 'standard_name' : 'time', 'calendar' : 'Gregorian'}

    #x
    out['x'] = inp.x.values
    out['x'].attrs = {'units' : 'm', 'standard_name' : 'x'}

    #y
    out['y'] = inp.y.values
    out['y'].attrs = {'units' : 'm', 'standard_name' : 'y'}


    #=============================
    ###Dynamics and thermodynamics
    #=============================

    #Sensible heat flux
    out['hfss'] = (['time','x','y'],(inp['SHF']).values)
    out['hfss'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upward_sensible_heat_flux', 'comment' : 'postive upward'}

    #Latent heat flux
    out['hfls'] = (['time','x','y'],(inp['LHF']).values)
    out['hfls'].attrs = {'units' : 'W m-2', 'standard_name' : 'surface_upward_latent_heat_flux', 'comment' : 'positive upward'}

    #Precipitable water vapor
    out['prw'] = (['time','x','y'],(inp['PW']).values)   # from mm to kg m-2
    out['prw'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmospheric_water_vapor_content'}

    #Surface zonal wind
    out['us'] = (['time','x','y'],(inp['USFC']).values)
    out['us'].attrs = {'units' : 'm s-1', 'standard_name' : 'surface_eastward_wind'}

    #Surface meridional wind
    out['vs'] = (['time','x','y'],(inp['VSFC']).values)
    out['vs'].attrs = {'units' : 'm s-1', 'standard_name' : 'surface_northward_wind'}


    #====================================
    ###Aerosol, clouds, and precipitation
    #====================================

    #Surface precipitation flux
    out['pr'] = (['time','x','y'],(inp['Prec']).values/24/3600)
    out['pr'].attrs = {'units' : 'kg m-2 s-1', 'standard_name' : 'precipitation_flux_at_surface'}

    #CWP
    out['cwp'] = (['time','x','y'],(inp['QCPATH']).values)  # NOAA SAM variable name: CWP
    out['cwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_cloud_water', 'comment' : 'cloud water only'}

    #LWP
    out['lwp'] = (['time','x','y'],(inp['QCPATH']+inp['QRPATH']).values)
    out['lwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_liquid_water', 'comment' : 'both cloud and rain water'}

    #RWP
    out['rwp'] = (['time','x','y'],(inp['QRPATH']).values)  # NOAA SAM variable name: RWP
    out['rwp'].attrs = {'units' : 'kg m-2', 'standard_name' : 'atmosphere_mass_content_of_rain_water'}

    #Cloud top height
    out['zct'] = (['time','x','y'],(inp['ZC']).values)  # NOAA SAM variable name: zct
    out['zct'].attrs = {'units' : 'm', 'standard_name' : 'cloud_top_altitude'}


    # Currently, These variables are not in outputs of UW SAM

    # #Inversion height
    # out['zi'] = (['time','x','y'],(inp['zi']).values)
    # out['zi'].attrs = {'units' : 'm', 'standard_name' : 'inversion_height'}

    # #Cloud optical depth
    # out['optc'] = (['time','x','y'],(inp['QCLOPD']).values)
    # out['optc'].attrs = {'units' : '1', 'standard_name' : 'atmosphere_optical_thickness_due_to_cloud'}

    # #Rain optical depth
    # out['optr'] = (['time','x','y'],(inp['QPLOPD']).values)
    # out['optr'].attrs = {'units' : '1', 'standard_name' : 'atmosphere_optical_thickness_due_to_rain'}

    # #Cloud base height
    # out['zcb'] = (['time','x','y'],(inp['zcb']).values)
    # out['zcb'].attrs = {'units' : 'm', 'standard_name' : 'cloud_base_altitude'}

    # #Cloud droplet number at cloud top
    # out['ncct'] = (['time','x','y'],(inp['ncct']).values)
    # out['ncct'].attrs = {'units' : 'kg-1', 'standard_name' : 'number_of_cloud_droplets_at_cloud_top'}

    # #Cloud droplet effective radius at cloud top
    # out['cerct'] = (['time','x','y'],(inp['cerct']).values)
    # out['cerct'].attrs = {'units' : 'm', 'standard_name' : 'cloud_droplet_effective_radius_at_cloud_top'}


    #===========
    ###Radiation
    #===========

    #TOA incoming SW
    out['rsdt'] = (['time','x','y'],(inp['SOLIN']).values)
    out['rsdt'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_incoming_shortwave_flux'}

    #TOA outgoing SW
    out['rsut'] = (['time','x','y'],(inp['SOLIN']-inp['SWNT']).values)
    out['rsut'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_shortwave_flux'}

    #TOA outgoing LW
    out['rlut'] = (['time','x','y'],(inp['LWNT']).values)
    out['rlut'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_longwave_flux'}

    #TOA outgoing clear-sky SW
    out['rsutcs'] = (['time','x','y'],(inp['SOLIN']-inp['SWNTC']).values)
    out['rsutcs'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_shortwave_flux_assuming_clear_sky'}

    #TOA outgoing clear-sky LW
    out['rlutcs'] = (['time','x','y'],(inp['LWNTC']).values)
    out['rlutcs'].attrs = {'units' : 'W m-2', 'standard_name' : 'toa_outgoing_longwave_flux_assuming_clear_sky'}


    #==========
    ###Metadata
    #==========

    out.attrs['case'] = 'SEA_STARR/'+caseid
    out.attrs['title'] = '2D output for SEA_STARR/%s case' % caseid
    out.attrs['reference'] = 'Diamond et al. (in prep), ACPD'
    out.attrs['author'] = 'Michael Diamond (michael.diamond@noaa.gov) and Ehsan Erfani (Erfani@uw.edu)'
    out.attrs['version'] = 'Created on ' + year + '-' + month + '-' + day
    out.attrs['format_version'] = 'DEPHY SCM format version 1'
    out.attrs['modifications'] = ''
    out.attrs['script'] = 'DEPHY_2D.ipynb'


    #===========
    ###Save file
    #===========

    os.system('rm %s' % output_path) #Clear old file if it exists
    out.to_netcdf(output_path)

In [4]:
input_path  = '../OUT_2D/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_CTRL_LD_384.2Dbin_1.nc'   
output_path = './SEA_STARR_CTRL_2D_SAM_UW.nc'
caseid      ='CTRL'

create_DEPHY_2Doutput(input_path,output_path,caseid)

In [6]:
input_path  = '../OUT_2D/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_NA100_LD_384.2Dbin_1.nc'   
output_path = './SEA_STARR_NA100_2D_SAM_UW.nc'
caseid      ='NA100'

create_DEPHY_2Doutput(input_path,output_path,caseid)

In [4]:
input_path  = '../OUT_2D/SEA_STARR_FAST_384sqx288_50m_M2005PA_RRTM4PBL_UM5_ProgAer_NA30_LD_384.2Dbin_1.nc'   
output_path = './SEA_STARR_NA30_2D_SAM_UW.nc'
caseid      ='NA30'

create_DEPHY_2Doutput(input_path,output_path,caseid)